# AI16 / 회의록 요약문 생성 모델 (심지은)

- TextRank와 KoBART 모델을 사용하여 녹취된 회의록에 대한 요약문을 생성하는 Text Summarization Project입니다.


## **0. Package**
- Package_processing : kss(문장분리기)
- Package_TextRank : lovit/Textrank
- c.f. KoBART 패키지는 하단 

### **Package_processing**

In [ ]:
# 데이터 전처리 / 문장분리기 #5분 
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 KB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 KB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 54.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kss: filename=kss-4.5.1-py3-none-any.whl size=53218 sha256=edd71cb6de587d35c3c4c5683f8a260c2fe0a125c9efcd230ea6e648375cd4c0
  Stored in directory: /root/.cache/pip/wheels/8c/f9/05/a93eb0defc9eed494ee4d42e3d9076fade98f08982f41cf09f
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646667 sha256=4744c14280ac89c08e54c23a916a0ccc1393e98209d74ac42ba5c6400ec1f4a3
  Stored in directory: /root/.cache/pip/wheels/a9/98/a3/085a909aa25805f321467a3af3b80d43b6094ed335b4818775
Successfully built kss pecab


### **Package_TextRank**

In [ ]:
# TextRank 패키지 설치 
!pip install konlpy
!pip install git+https://github.com/lovit/textrank.git

!git clone https://github.com/lovit/textrank.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/lovit/textrank.git to /tmp/pip-req-build-bhu86ysx
  Running command git clone --filter=blob:none --quiet https://github.com/lovit/textrank.git /tmp/pip-req-build-bhu86ysx
  Resolved https://github.com/lovit/textrank.git to commit 578c7fbbb22b24561b1bd4bab9a9ecc2dc9515e1
  Preparing metadata (setup.py) ... done
fatal: destination path 'textrank' already exists and is not an empty directory.


## **1. Data Load & Processing**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/project4/RawData

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/project4/RawData


In [ ]:
import json

with open("train.json", "r") as st_json:
    sents = json.load(st_json)

#### **Data info**

In [ ]:
# 데이터 구성 
sents[0]

{'id': 1000,
 'title': '제207회 완주군의회(임시회) 제 1 차 본회의회의록',
 'region': '완주',
 'num_agenda': 4,
 'context': {'AGENDA_1': {'0': '의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제1차 본회의 개의를 선포합니다. 먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 주시기 바랍니다.',
   '1': '의사팀장 이은미 입니다. 의회관련 사항을 보고 드리겠습니다. 먼저 제207회 완주군의회 임시회 소집경위를 보고 드리겠습니다. 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 의안접수 및 회부사항입니다. 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 보호 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 다음은 완주군수로부터 제출된 안건입니다. 삼례문화예술촌 설치 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 자세한 내용은 배부해 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다.'},
  'AGENDA_2': {'0': '의사팀장 수고하셨습니다. 먼저 의사일정 제1항 제207회 완주군의회 임시회 회기 결정의 건을 상정합니다. 제207회 완주군의회 임시회 회기 결정의 건에 대하여는 서남용 의원님 외 4인의 의원님이 발의한대로 8월 26일부터 9월 4일까지 10일간의 회기를 결정하고자 합니다. 의원 여러분 이의 있으십니까?',
   '1': '(『없습니다』하는 의원 있음)',
   '2': '이의가 없으므로 가결되었음을 선포합니다.'},
  'AGENDA_3': {'0': '다음은 의사일정 제2항 제207회 완주군의회 

In [ ]:
# 데이터 총개수
len(sents)

463

In [ ]:
sents[0]['context']['AGENDA_1'].values()

dict_values(['의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제1차 본회의 개의를 선포합니다. 먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 주시기 바랍니다.', '의사팀장 이은미 입니다. 의회관련 사항을 보고 드리겠습니다. 먼저 제207회 완주군의회 임시회 소집경위를 보고 드리겠습니다. 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 의안접수 및 회부사항입니다. 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 보호 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 다음은 완주군수로부터 제출된 안건입니다. 삼례문화예술촌 설치 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 자세한 내용은 배부해 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다.'])

#### **Data Preprocessing**

In [ ]:
# 데이터프레임 생성 
import pandas as pd

df = pd.DataFrame(columns=['id','agenda','text', 'label']) #7초 

for i in range(len(sents)):
  id = sents[i]['id']
  agenda = list(sents[i]['context'].keys())
  for j in agenda:
    text = list(sents[i]['context'][j].values())
    label = sents[i]['label'][j]['summary']
    df = df.append({'id':id, 'agenda':j, 'text':text, 'label':label}, ignore_index=True)

In [ ]:
df.tail()

,id,agenda,text,label
2989,1462,AGENDA_6,[회계과장입니다. 회계과 소관 2020년도 주요 현안사업을 보고드리겠습니다. 보고드...,음성읍행정복지센터 앞에 사유지 매입은 필요한 사업이기에 예산을 다시 반영해서라도 매...
2990,1462,AGENDA_7,[민원과장 김후식입니다. 민원과 소관 2020년도 주요 현안사업 추진상황을 보고드리...,공공시설물 전산화 작업시에 사유시설과 같이 협력해서 처리할 수 있도록 할 것. 지적...
2991,1462,AGENDA_8,[경제과장 박세덕입니다. 경제과 2020년 주요 현안사업에 대하여 보고드리겠습니다....,용계리 지역의 땅꺼짐 현상에 대해 군 차원의 대처를 위한 고민을 해줄 것. 생극면 ...
2992,1462,AGENDA_9,[농정과장 김기연입니다. 농정과 소관 2020년도 하반기 주요 현안사업을 보고드리겠...,음성군에서 생산된 쌀이 진천으로 수매하는 현상이 발생하고 있는데 음성군 브랜드를 활...
2993,1462,AGENDA_10,[축산식품과장 송요성입니다. 축산식품과 소관 2020년도 군정주요 현안사업을 보고드...,현재 저조한 길고양이 중성화사업에 적극행정을 취해줄 것. 친환경 로컬푸드 매장 조성...


In [ ]:
df.text[0]

['의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제1차 본회의 개의를 선포합니다. 먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 주시기 바랍니다.',
 '의사팀장 이은미 입니다. 의회관련 사항을 보고 드리겠습니다. 먼저 제207회 완주군의회 임시회 소집경위를 보고 드리겠습니다. 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 의안접수 및 회부사항입니다. 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 보호 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 다음은 완주군수로부터 제출된 안건입니다. 삼례문화예술촌 설치 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 자세한 내용은 배부해 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다.']

In [ ]:
# text 화자별 합치기 / KoBART용
def concatsentence(lst):
  text = str()
  for i in range(len(lst)):
    text_add = lst[i].replace('.','. ')
    text = text + text_add
  return text

df['text_concat'] = df.text.apply(concatsentence)
df['text_concat'] = df['text_concat'].replace('  ',' ')

In [ ]:
# text 문장단위 분리 / TextRank용
from kss import split_sentences

def splitsentence(text): #5분
  lst_split = []
  for sent in split_sentences(text):
    lst_split.append(str(sent))
  return lst_split

df['text_split'] = df['text_concat'].apply(splitsentence) 


In [ ]:
# 요약문 label 글자수 확인 
df['len_label'] = df['label'].apply(lambda x:len(x))

In [ ]:
# 요약문이 너무 짧거나 긴 데이터 확인
df.describe()

,len_label
count,2994.000000
mean,81.472612
std,68.617648
min,7.000000
25%,34.000000
50%,51.000000
75%,114.000000
max,742.000000


In [ ]:
df[df['len_label'] < 15]['label']
df[df['len_label'] > 500]['label'][1326]

'경제환경위원회 김태수 의원이 충북도가 실제로는 15조 원의 투자 유치임에도 반도체 클러스터 투자 발표 기자회견 등을 통해 50조 원의 투자를 유치했다고 과대포장한 사실을 지적함. 도시건설위원회 홍성각 의원이 시민들의 무작위 무기명 설문조사를 통해 철거 의견을 묻고, 3분의 2 이상의 철거 의견이 있을 시에는 시장과 문화재청장 간의 대담을 통해 시청 본관 존치를 위한 결론을 재고할 것을 요구함. 농업정책위원회 전규식 의원이 도농복합도시로서 도농 교류 활성화를 위한 도농상생팀과 농촌동의 적극적인 업무 추진 등 체계적인 농업행정 시행을 위한 농촌동 산업팀을 신설하고 적정 수준의 인력을 배치할 것을 요청함. 복지교육위원회 최충진 의원이 시의 청소년정책이 학교 밖, 가정 밖 청소년의 보호 정책에서 나아가 교육지원청 등을 비롯한 다양한 기관 및 단체와 연계하여 능동적이고 종합적인 정책이 되어야 함을 요구함. 김성택 의원이 사회적 갈등 해소를 위하여 일정 규모 이상의 허가민원을 이해관계자들에게 사전에 예고하는 허가민원 사전예고제 실시와 관련 조례 제정, 제정 이전까지의 행정지침 마련 및 시행을 촉구함.'

In [ ]:
# 요약문이 너무 짧거나 긴 데이터 제거 
df = df[(df['len_label'] > 15) & (df['len_label'] < 500)]

# 그외 오타 제거
df.loc[1597, 'text_split'][7] = df.loc[1597, 'text_split'][7].replace('\U000f0850','')

In [ ]:
df.to_csv('original.csv', encoding='utf-8')

## **2. Text Summarization**

- https://github.com/uoneway/Text-Summarization-Repo Text summarization task에 사용되는 모델 및 알고리즘 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/project4/RawData

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/project4/RawData


In [ ]:
# 가공 후 데이터 / 전처리 생략 
import pandas as pd
df = pd.read_csv('original.csv', encoding='utf-8', index_col=0)

In [ ]:
# 데이터 스플릿
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state=42)
train, val = train_test_split(train, random_state=42)
train.shape, val.shape, test.shape

((1674, 7), (559, 7), (745, 7))

In [ ]:
train.head()

# id 각 회의의 고유번호
# agenda 각 회의에서 진행된 안건의 고유번호
# text 발화내용 / 화자별
# text_concat 발화내용 / 총합
# text_split 발화내용 / 문장별 
# label 요약문(target)
# len_label 요약문의 글자 수 

,id,agenda,text,label,text_concat,text_split,len_label
2749,1423,AGENDA_3,"['의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다.', '제188회 음성...","제188회 임시회 회의록 서명의원으로 반광홍 의원, 정태완 의원이 선출됨.","의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제188회 음성군의회 임...","['의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다.', '제188회 음성...",41
898,1148,AGENDA_5,"['의사일정 제3항, 밥쌀용 쌀 수입 반대 결의안을 상정합니다.', '대표발의하신 ...",밥쌀용 쌀 수입 반대 결의안은 정부가 밥쌀용 쌀 3만 톤을 미국과 중국에서 수입하기...,"의사일정 제3항, 밥쌀용 쌀 수입 반대 결의안을 상정합니다. 대표발의하신 이상정 의...","['의사일정 제3항, 밥쌀용 쌀 수입 반대 결의안을 상정합니다.', '대표발의하신 ...",89
1366,1216,AGENDA_5,"['의사일정 제4항, 음성군 지방공무원 정원조례 일부개정조례안, 의사일정 제5항, ...",음성군 지방공무원 정원조례 일부개정조례안은 전문위원의 직급과 정수 기준이 개정되어 ...,"의사일정 제4항, 음성군 지방공무원 정원조례 일부개정조례안, 의사일정 제5항, 음성...","['의사일정 제4항, 음성군 지방공무원 정원조례 일부개정조례안, 의사일정 제5항, ...",240
1927,1304,AGENDA_4,"['의사일정 제3항, 2006년도 제2회 세입·세출 추가경정예산안 및 2006년도 ...","2006년도 제2회 세입, 세출 추가경정예산안 및 2006년도 기금운용계획변경계획안...","의사일정 제3항, 2006년도 제2회 세입·세출 추가경정예산안 및 2006년도 기금...","['의사일정 제3항, 2006년도 제2회 세입·세출 추가경정예산안 및 2006년도 ...",52
1177,1190,AGENDA_9,['산림녹지과장입니다. 2018년도 행정사무감사 시 시정ㆍ건의사항에 대한 조치결과 ...,만세기념비 등이 있는 오류리 공원의 화장실을 관리하여 주민들이 실제로 이용할 수 있...,산림녹지과장입니다. 2018년도 행정사무감사 시 시정ㆍ건의사항에 대한 조치결과 및...,"['산림녹지과장입니다.', '2018년도 행정사무감사 시 시정ㆍ건의사항에 대한 조치...",55


In [ ]:
# KoBART 학습용
df_train = train[['text_concat','label']]
df_val = val[['text_concat','label']]
df_test = test[['text_concat','label']]

#### **TextRank**

In [ ]:
# textrank / lovit textrank 모듈 
## extractive approaches / 핵심문장 추출 = 요약 / 비지도학습 
# 문장단위 나누기 전처리 필요 

from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer

komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3, #유사도 0.3이상일때 
    verbose = False
)

sents = [
  '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다', 
  '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다', 
  '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다', 
  '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다', 
  '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다', 
  '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다', 
  '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다', 
  '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기', 
  '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다', 
  '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다', 
  '김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다', 
  '김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다', 
  '머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다', 
  '성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다', 
  '총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다', 
  '총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다', 
  '성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다', 
  '성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다', 
  '경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다', 
  '일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다'
]

keysents = summarizer.summarize(sents, topk=3) #문장개수 
keysents

[(0,
  1.7010195551998453,
  '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다'),
 (7,
  1.4886043194936454,
  '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기'),
 (8,
  1.4278590385659577,
  '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다')]

- https://mkjjo.github.io/python/2019/07/09/korean_preprocessing.html 워드클라우드 

###### **TextRank용 전처리**

In [ ]:
# 데이터 전처리 / TextRank
test_textrank = test.copy()
test_textrank['text_split'] = test_textrank['text_split'].apply(lambda x: x.replace('\n','').replace('\t','').replace('\r',''))
test_textrank['text_split'] = test_textrank['text_split'].apply(lambda x: x.replace('『','').replace('』','').replace('(','').replace(')','').replace('?','').replace('「','').replace('」',''))
test_textrank['text_split'] = test_textrank['text_split'].apply(lambda x: x.replace('\\U000f0852','').replace('\\U000f0853','').replace('\\U000f0850','').replace('\\U000f0851',''))

In [ ]:
type(test_textrank['text_split'][2420])

str

In [ ]:
%pwd

'/content/drive/MyDrive/project4/RawData/KoBART-summarization'

In [ ]:
# komoran = Komoran(userdic = './dict.txt') # userdic = './dict.txt')
# content = test_textrank['text_split'][2420]
# komoran.pos(content)

[('[', 'SS'),
 ("'", 'SS'),
 ('의사일정', 'NNG'),
 ('제', 'XPN'),
 ('3', 'SN'),
 ('항', 'NNG'),
 ('의사일정', 'NNG'),
 ('변경', 'NNG'),
 ('의', 'JKG'),
 ('건', 'NNB'),
 ('을', 'JKO'),
 ('상정', 'NNG'),
 ('하', 'XSV'),
 ('ㅂ니다', 'EF'),
 ('.', 'SF'),
 ("'", 'SS'),
 (',', 'SP'),
 ("'", 'SS'),
 ('당초', 'NNG'),
 ('7월 3일', 'NNP'),
 ('제', 'XPN'),
 ('2', 'SN'),
 ('차', 'NNB'),
 ('본회의', 'NNG'),
 ('시', 'XSN'),
 ('상임위원회', 'NNG'),
 ('위원', 'NNG'),
 ('선임', 'NNG'),
 ('의', 'JKG'),
 ('건', 'NNB'),
 ('과', 'JKB'),
 ('상임', 'NNG'),
 ('위원장', 'NNG'),
 ('선거', 'NNG'),
 ('를', 'JKO'),
 ('4', 'SN'),
 ('항', 'NNG'),
 ('과', 'JC'),
 ('5', 'SN'),
 ('항', 'NNG'),
 ('으로', 'JKB'),
 ('상정', 'NNG'),
 ('하', 'XSV'),
 ('아', 'EC'),
 ('처리', 'NNG'),
 ('하', 'XSV'),
 ('고자', 'EC'),
 ('하', 'VV'),
 ('는데', 'EC'),
 ('의원', 'NNG'),
 ('여러분', 'NNG'),
 ('이의', 'NNG'),
 ('없', 'VA'),
 ('으시', 'EP'),
 ('ㅂ니까', 'EC'),
 ("'", 'SS'),
 (',', 'SP'),
 ("'", 'SS'),
 ('없', 'VA'),
 ('습니다', 'EC'),
 ('하', 'VX'),
 ('는', 'ETM'),
 ('의원', 'NNG'),
 ('있', 'VV'),
 ('음', 'ETN'),
 ('.', 'S

- https://komorandocs.readthedocs.io/ko/latest/manual/manual.html Komoran 사용자 사전 API
- https://steadiness-193.tistory.com/144 Komoran 사용자사전 예제 

In [ ]:
import ast
summarizer.summarize(ast.literal_eval(test_textrank['text_split'][2420]), topk=3)

[(1,
  1.6682934603175785,
  '당초 7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5항으로 상정하여 처리하고자 하는데 의원여러분 이의 없으십니까'),
 (4, 0.948651844970584, '다음은 상임위원회위원 선임과 상임위원장 선거입니다만'),
 (3, 0.9438726586826501, '이의가 없으므로 가결되었음을 선포합니다.')]

In [ ]:
test.head()

,id,agenda,text,label,text_concat,text_split,len_label
2420,1377,AGENDA_4,['의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시...,제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...,의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시 ...,"['의사일정 제3항 의사일정변경의 건을 상정합니다.', '당초 7월 3일 제2차 본...",55
2866,1444,AGENDA_4,"['의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다.', '...",음성군 지역치안협의회 조례 일부개정조례안은 지역치안협의회에 보조금 지원이 용이하도록...,"의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다. 행정과장님...","['의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다.', '...",78
1724,1272,AGENDA_5,['다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제10...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,['다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제10...,47
1124,1183,AGENDA_4,['다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다....,예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...,다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다. ...,['다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다....,164
975,1157,AGENDA_3,"['다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3...",완주군 지방공무원정원 조례 일부개정조례안은 부동산 실거래가 신고제 등 토지관련 업무...,"다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3항 ...","['다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3...",368


In [ ]:
train.loc[2031, :]

id                                                          1319
agenda                                                  AGENDA_1
text           [의석을 정돈하여 주시기 바랍니다., 성원이 되었으므로 지금부터 제314회 음성군의...
label                             제314회 음성군의회 임시회 제3차 본회의 개의 선포.
text_concat    의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 제314회 음성군의회 ...
text_split     [의석을 정돈하여 주시기 바랍니다., 성원이 되었으므로 지금부터 제314회 음성군의...
len_label                                                     30
Name: 2031, dtype: object

In [ ]:
test_textrank['text_split'][774]

"['의사일정 제6항, 동서고속도로IC위치변경촉구건의안을 상정합니다.', '발의하신 최관식 의원님 나오셔서 제안 설명하여 주시기 바랍니다.', '최관식 의원입니다.', '동서고속도로IC위치변경촉구건의안을 채택하여 대통령, 국무총리, 건설교통부 장관, 국회 의장, 새천년민주당 대표, 한나라당 총재, 자유민주연합 총재에게 건의하고자 합니다.', '제안이유로는 우리 음성 지역 주민들은 동서 고속도로가 신설되는 것을 환영하면서도 지역발전의 관건인 IC위치를 결정함에 있어서 지역주민들의 의견을 수렴하고 지역발전과 국토의 균형발전을 충분히 고려하여 결정하여야 함에도 지역 주민의 의사를 무시하고 결정하려는 관계 당국의 처사에 실망감을 감추지 못하고 있으며, 국토를 균형 있게 개발하려는 군민들의 뜻을 모아 지역이 골고루 발전할 수 있는 필요한 조치를 강구하여 줄 것을 건의하고자 합니다.', '그러면 건의안을 낭독하겠습니다.', '평소 국토의 균형 발전과 편리하고 살기 좋은 새로운 교통 환경 조성을 위해서 항상 재대한 관심을 갖고 노력하시는 대통령, 국회 의장, 새천년민주당 대표, 한나라당 총재, 자유민주연합 총재, 국무총리, 건설교통부 장관님의 노고에 대해서 충북 음성 군민과 음성군의회 의원 일동은 깊은 감사를 드리고 있습니다.', '우리 음성군 지역은 충북과 경기도의 접경지역에 위치해 있어서 낙후된 지역이었으나, 중부 고속도로의 개통과 더불어 신공업 지역으로 급부상하고 있는 지역입니다.', '또한 우리 지역 여주～충주 구간을 통과하는 중부 내륙고속도로가 내년 말 완공목표로 공사 중에 있으며, 평택 안중～삼척간 동서 고속도로가 2006년 개통을 목표로 내년도에 본격 착공할 계획으로 있어서 주민들은 지역발전에 대하여 지대한 관심과 큰 기대를 갖고 있습니다.', '고속도로 통과 지역의 발전을 위해서는 IC가 어디에 위치하느냐가 지역발전의 크나큰 관건이라고 할 수 있을 것입니다.', '따라서 지역 주민들은 고속도로가 개통되는 것을 환영하면서도 정작 지역발전의 관건인 IC 위치를 

###### **TextRank 적용**

In [ ]:
# TextRank 적용 / 45초 
import ast

test_textrank['textrank'] = ''
for num_string in list(test_textrank.index):
  print(num_string)
  print(test_textrank['text_split'][num_string])
  if len(ast.literal_eval(test_textrank['text_split'][num_string])) > 3:
    keysents = summarizer.summarize(ast.literal_eval(test_textrank['text_split'][num_string]), topk=3) #str이 아닌 list 형태로 인식하도록 
    test_textrank.at[num_string, 'textrank'] = keysents
  else :
    keysents = test_textrank['text_split'][num_string]
    test_textrank.at[num_string, 'textrank'] = keysents

2420
['의사일정 제3항 의사일정변경의 건을 상정합니다.', '당초 7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5항으로 상정하여 처리하고자 하는데 의원여러분 이의 없으십니까', '없습니다하는 의원 있음. ', '이의가 없으므로 가결되었음을 선포합니다.', '다음은 상임위원회위원 선임과 상임위원장 선거입니다만', '위원 선임을 하여야 하기 때문에 잠시 정회를 선포합니다.']
2866
['의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다.', '행정과장님께서는 나오셔서 본 안건에 대하여 제안설명해 주시기 바랍니다.', '행정과장입니다.', '음성군 지역치안협의회 조례 일부개정조례안에 대해서 설명을 드리겠습니다.', '개정이유는 법질서 확립과 안전하고 행복한 음성군을 만들고자 설치된 지역치안협의회에 보조금 지원이 용이하도록 관련 조항을 개정하고자 하는 사항입니다.', '주요내용은 안 제10조에 제목을 보조금 지원 등을 지원 등으로 하고, 같은 조 제1항에 예산의 범위 내에서 사회단체보조금 지원 조례에 따라 되어 있는 것을 예산의 범위에서 지원하는 것으로 변경하는 사항이 되겠습니다.', '네 번째, 조례개정안과 신ㆍ구조문대비표는 붙임을 참고해 주시고요, 관계법령은 해당 사항이 없습니다.', '일곱 번째, 사전협의 및 승인신청 결과입니다.', '성별영향평가 검토결과는 해당 사항이 없습니다.', '보조금 지원 관련 검토결과도 특이의견이 없었습니다.', '지난 10월 24일 조례ㆍ규칙심의회에서 원안 가결이 되었습니다.', '여덟 번째, 비용추계서 및 첨부제외 사유서는 붙임을 참고하여 주시기 바랍니다.', '아홉 번째 입법예고 결과는 9월 26일부터 10월 16일까지 입법예고를 하였습니다만 제출된 의견이 없었습니다.', '마지막으로 제안자 의견입니다.', '종전의 음성군 사회단체보조금 지원 조례에 따라 보조금 지원은 규정과 절차상 필요 시 경비지원이 어렵기 때문에 이를 개선하기 위해서 본 조례를 개정하고자 하는 

In [ ]:
# 너무 짧은 문장들
train[train['textrank'] == ''].shape #(75, 8)
train['text_split'][2031]

['의석을 정돈하여 주시기 바랍니다.', '성원이 되었으므로 지금부터 제314회 음성군의회 임시회 제3차 본회의를 개의하겠습니다.']

In [ ]:
# 짧은 문장들에 에러 발생 
summarizer.summarize(train['text_split'][2031], topk=1)

/usr/local/lib/python3.8/dist-packages/scipy/sparse/sputils.py:114: UserWarning: object dtype is not supported by sparse matrices
  warnings.warn("object dtype is not supported by sparse matrices")


ValueError: ignored

In [ ]:
test_textrank.head()

,id,agenda,text,label,text_concat,text_split,len_label,textrank
2420,1377,AGENDA_4,['의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시...,제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...,의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시 ...,"['의사일정 제3항 의사일정변경의 건을 상정합니다.', '당초 7월 3일 제2차 본...",55,"[(1, 1.6682934603175785, 당초 7월 3일 제2차 본회의시 상임위..."
2866,1444,AGENDA_4,"['의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다.', '...",음성군 지역치안협의회 조례 일부개정조례안은 지역치안협의회에 보조금 지원이 용이하도록...,"의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다. 행정과장님...","['의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다.', '...",78,"[(29, 1.4717813659629004, 의사일정 제3항, 음성군 지역치안협의..."
1724,1272,AGENDA_5,['다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제10...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,['다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제10...,47,"[(0, 1.2249777916398097, 다음은 지방자치법 제64조제2항과 회의..."
1124,1183,AGENDA_4,['다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다....,예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...,다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다. ...,['다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다....,164,"[(4, 1.4170457588599894, 존경하는 의장님, 그리고 동료 의원 여..."
975,1157,AGENDA_3,"['다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3...",완주군 지방공무원정원 조례 일부개정조례안은 부동산 실거래가 신고제 등 토지관련 업무...,"다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3항 ...","['다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3...",368,"[(18, 1.4308450362205591, 다음은 의사일정 제3항 완주군별정직지..."


In [ ]:
# 최종 summary 칼럼으로 가공 
for i in test_textrank.index:
  print(i)
  if test_textrank.loc[i, 'textrank'] == '':
    test_textrank.loc[i, 'summary'] = ast.literal_eval(test_textrank.loc[i, 'text_split'])[0]
  elif len(test_textrank.loc[i, 'text_split']) >= 3:
    sents = test_textrank.loc[i, 'textrank'][0][2]
    test_textrank.loc[i, 'summary'] = sents
  else:
    test_textrank.loc[i, 'summary'] = ast.literal_eval(test_textrank.loc[i, 'text_split'])[0]

2420
2866
1724
1124
975
2945
468
2560
891
2226
1929
794
1814
829
93
1272
849
1792
1827
145
2513
2761
677
1356
2773
193
441
63
2205
2471
2699
2130
692
807
425
2373
2153
500
2040
1566
822
2355
2187
2285
1001
1443
212
2027
2485
2449
768
2113
1735
2421
1172
175
52
1468
1881
334
1794
1371
45
2843
1556
2095
179
103
753
685
511
1897
0
1920
292
1834
1271
889
2950
2081
357
335
324
462
560
2231
2044
751
1211
1638
415
1386
821
1737
2287
2345
549
204
371
2086
537
195
1790
2828
1460
1610
1157
51
2671
2663
2221
700
1393
2379
608
1084
2024
1987
2202
532
1190
416
2495
1504
1441
2200
1725
1604
609
969
2540
1064
2697
2372
67
2497
2368
197
1671
1239
1901
2496
745
702
2237
955
419
707
2780
125
1545
750
1346
1898
1531
2807
1010
1388
2390
2595
1507
1600
2409
2549
1009
1389
512
1993
2672
2366
1114
2122
1105
1349
1110
80
32
249
1378
1893
1255
1329
2244
2930
1418
2014
2927
88
1419
1805
2307
2855
2840
1577
1252
2634
2902
790
1284
1342
142
2700
1104
1170
1447
1863
1674
176
1761
2717
1642
540
1338
2923
2177
2466


In [ ]:
test_textrank[['summary','label']]

,summary,label
2420,당초 7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항...,제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...
2866,"의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 원안대로 의결하고자 하...",음성군 지역치안협의회 조례 일부개정조례안은 지역치안협의회에 보조금 지원이 용이하도록...
1724,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.
1124,"존경하는 의장님, 그리고 동료 의원 여러분!완주군의회 예산결산 특별위원회 구성 결의...",예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...
975,다음은 의사일정 제3항 완주군별정직지방공무원의범위에관한조례 일부개정조례안에 대하여 ...,완주군 지방공무원정원 조례 일부개정조례안은 부동산 실거래가 신고제 등 토지관련 업무...
...,...,...
1448,다음은 의사일정 제2항 98년도 제1회 추가경정 예산안을 상정합니다.,?98년도 제1회 추가경정 예산안 제안 설명.
1757,먼저 의사일정 제1항 제253회 완주군의회 임시회 회기 결정의 건을 상정합니다.,제253회 완주군의회 임시회 회기는 7월 14일부터 7월 23일까지 10일간으로 가결됨.
2276,다음은 의사일정 제2항 제252회 완주군의회 임시회 회의록 서명의원 선출의 건을 상...,제252회 완주군의회 임시회 회의록 서명의원으로 김재천 의원과 이인숙 의원이 선출됨.
662,"의사일정 제8항, 2021년도 수시분 공유재산 관리계획안을 상정합니다.",2021년도 수시분 공유재산 관리계획안 제안 설명.


In [ ]:
# TextRank SentenceSummarizer 결과저장 
test_textrank.to_csv('textrank.csv',encoding='utf-8')

In [ ]:
# TextRank Keyword Extraction
from textrank import KeywordSummarizer

keyword_summarizer = KeywordSummarizer(tokenize=komoran_tokenizer, min_count=2, min_cooccurrence=1)
keyword_summarizer.summarize(df['text'][0], topk=20)

[('조례/NNP', 2.238228053318344),
 ('완주군/NNP', 1.9758307719469501),
 ('의원/NNG', 1.5415162838626904),
 ('안/NNP', 1.5415162838626904),
 ('회/NNB', 1.2789391415583307),
 ('회부/NNG', 1.193160399134864),
 ('드리/VV', 1.193160399134864),
 ('등/NNB', 1.193160399134864),
 ('의회/NNP', 1.0160153412396702),
 ('사항/NNG', 0.9303727275338443),
 ('임시회/NNP', 0.9303727275338443),
 ('하/VV', 0.8448045144070372),
 ('해당/NNG', 0.8448045144070372),
 ('건/NNB', 0.8448045144070372),
 ('개정/NNG', 0.8448045144070372),
 ('일부/NNG', 0.8448045144070372),
 ('다음/NNG', 0.8448045144070372),
 ('관하/VV', 0.8448045144070372),
 ('안건/NNG', 0.8448045144070372),
 ('상임/NNG', 0.8448045144070372)]

- https://lovit.github.io/nlp/2019/04/30/textrank/ TextRank 알고리즘의 사용법 및 단점
- https://ebbnflow.tistory.com/292 TextRank 알고리즘 실제 구현과정
- KoBERT를 이용한 기계 독해 / 생성된 질문을 가지고 예측결과와 실제결과 

In [ ]:
# 소스코드 / ToyData 
# textrank / lovit textrank 모듈 
## extractive approaches / 핵심문장 추출 = 요약 / 비지도학습 
# 문장단위 나누기 전처리 필요 

from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer

komoran = Komoran()
def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3, #유사도 0.3이상일때 
    verbose = False
)

sents = [
  '오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다', 
  '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다', 
  '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다', 
  '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다', 
  '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다', 
  '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다', 
  '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다', 
  '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기', 
  '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다', 
  '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다', 
  '김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다', 
  '김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다', 
  '머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다', 
  '성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다', 
  '총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다', 
  '총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다', 
  '성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다', 
  '성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다', 
  '경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다', 
  '일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다'
]

keysents = summarizer.summarize(sents, topk=3) #문장개수 
keysents

#### **KoBART**

##### **Package_KoBART**

In [ ]:
# 패키지 설치 / KoBART # 총2분 
# !git clone https://github.com/seujung/KoBART-summarization.git
# %cd KoBART-summarization

# KoBART / SKT-AI
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart
!pip install transformers==4.16.2

!pip install -r requirements.txt

# 요구사양 
# pytorch>=1.10.0
# transformers==4.16.2
# pytorch-lightning==1.5.10
# streamlit==1.2.0


# %cd data

# !tar -zxvf train.tar.gz
# !tar -zxvf test.tar.gz

# %cd ../

# ! pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-rrnodpri/kobart_fd18bc32af624caf9e35c61210617d66
  Running command git clone --filter=blob:none --quiet https://github.com/SKT-AI/KoBART /tmp/pip-install-rrnodpri/kobart_fd18bc32af624caf9e35c61210617d66
  Resolved https://github.com/SKT-AI/KoBART to commit 30c5eb7b593828d6ec2d767eeedb2f2ed02c5c2a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.2/814.2 KB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.8/776.8 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 63.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6

- https://github.com/seujung/KoBART-summarization/blob/main/train.py TRAIN 파일 코드 

In [ ]:
# ImportError: cannot import name 'get_num_classes' from 'torchmetrics.utilities.data'
! pip install torchmetrics==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 KB 11.0 MB/s eta 0:00:00
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 0.11.1
    Uninstalling torchmetrics-0.11.1:
      Successfully uninstalled torchmetrics-0.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kobart 0.5.1 requires pytorch-lightning==1.2.1, but you have pytorch-lightning 1.3.8 which is incompatible.
kobart 0.5.1 requires torch==1.7.1, but you have torch 1.10.0 which is incompatible.
kobart 0.5.1 requires transformers==4.3.3, but you have transformers 4.8.2 which is incompatible.


In [ ]:
# 버전확인 
# !pip show torch

Name: torch
Version: 1.10.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.8/dist-packages
Requires: typing-extensions
Required-by: fastai, kobart, pytorch-lightning, torchaudio, torchmetrics, torchtext, torchvision


In [ ]:
# !pip show torchtext

Name: torchtext
Version: 0.14.1
Summary: Text utilities and datasets for PyTorch
Home-page: https://github.com/pytorch/text
Author: PyTorch core devs and James Bradbury
Author-email: jekbradbury@gmail.com
License: BSD
Location: /usr/local/lib/python3.8/dist-packages
Requires: numpy, requests, torch, tqdm
Required-by: 


In [ ]:
# torch 패키지 설치

#/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
# warn(f"Failed to load image Python extension: {e}")
# OSError: /usr/local/lib/python3.8/dist-packages/torchtext/_torchtext.so: undefined symbol
# >> 자신 torch 버전에 맞게 torchtext 재설치 >> https://github.com/pytorch/text#installation

!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchtext==0.8.0

# AttributeError: 'Trainer' object has no attribute '_data_connector'
! pip install pytorch_lightning==1.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.4/735.4 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 66.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.1+cu116
    Uninstalling torchvision-0.14.1+cu116:
      Successfully uninstalled torchvision-0.14.1+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 1.7.1+cu101 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you 

In [ ]:
# loader 오류 발생시 
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%pwd

'/content/drive/MyDrive/project4/RawData/KoBART-summarization'

In [ ]:
%cd ../

/content/drive/MyDrive/project4/RawData


##### **KoBART - Train**

In [ ]:
# 트레인 파일에 더하기
# news(data) summary(label) 두 칼럼 / 각 데이터는 str 형태 

%cd data

df_train.to_csv("train.tsv", mode='a', header=0, sep='\t', index=False)
df_val.to_csv("test.tsv", mode='a', header=0, sep='\t', index=False)

%cd ../

/content/drive/MyDrive/project4/RawData/KoBART-summarization/KoBART-summarization/data
/content/drive/MyDrive/project4/RawData/KoBART-summarization/KoBART-summarization


In [ ]:
# 데이터 학습
!python train.py  --gradient_clip_val 1.0 --max_epochs 5 --default_root_dir logs  --gpus 1 --batch_size 4 --num_workers 4

# 총 소요시간 9시간30분(에포크 0회)

Traceback (most recent call last):
  File "train.py", line 4, in <module>
    import numpy as np
  File "/usr/local/lib/python3.8/dist-packages/numpy/__init__.py", line 162, in <module>
    from . import ctypeslib
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 939, in get_code
  File "<frozen importlib._bootstrap_external>", line 1038, in get_data
KeyboardInterrupt
^C


In [ ]:
%pwd

'/content/drive/MyDrive/project4/RawData'

In [ ]:
%cd KoBART-summarization
# /content/drive/MyDrive/project4/RawData/KoBART-summarization

/content/drive/MyDrive/project4/RawData/KoBART-summarization


##### **model**
  - 직접 학습
  - digit82/kobart-summarization
  - gogamza/kobart-base-v2
  - gogamza/kobart-summarization

##### **tokenizer**
  - digit82/kobart-summarization
  - gogamza/kobart-base-v2 == gogamza/kobart-summarization (MyModel에서 동일한 결과 도출) 

  - c.f. v1은 사용불가

##### **KoBART - Application**

In [ ]:
# 학습한 체크포인트 불러오기
# OES 발생 시 다시 한번 torch 패키지 설치(미작동되는 경우 있음)

# !python get_model_binary.py --hparams ./logs/tb_logs/default/version_0/hparams.yaml --model_binary ./logs/model_chp/epoch=50-val_loss=1.464.ckpt
# !python get_model_binary.py --hparams ./logs/tb_logs/default/version_0/hparams.yaml --model_binary $(ls -1 -t ./logs/model_chp | head -1)
!python get_model_binary.py --hparams ./logs/tb_logs/default/version_0/hparams.yaml --model_binary ./logs/model_chp/epoch=00-val_loss=1.312.ckpt

get_model_binary.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)
Downloading: 100% 1.36k/1.36k [00:00<00:00, 1.62MB/s]
Downloading: 100% 496M/496M [00:05<00:00, 95.5MB/s]
Downloading: 100% 4.00/4.00 [00:00<00:00, 3.62kB/s]
Downloading: 100% 111/111 [00:00<00:00, 120kB/s]
Downloading: 100% 682k/682k [00:00<00:00, 2.18MB/s]


In [ ]:
# 코랩 끊김방지 코드 
# function ClickConnect(){
#     console.log("코랩 연결 끊김 방지"); 
#     document.querySelector("colab-toolbar-button#connect").click() 
# }
# setInterval(ClickConnect, 60 * 1000)

In [ ]:
# huggingface의 transformers BartModel 작성형태 
# from transformers import PreTrainedTokenizerFast, BartModel

# tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')
# model = BartModel.from_pretrained('gogamza/kobart-base-v2')

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

In [ ]:
df_train.head()

,text_concat,label
2749,"의사일정 제2항, 회의록 서명의원 선출의 건을 상정합니다. 제188회 음성군의회 임...","제188회 임시회 회의록 서명의원으로 반광홍 의원, 정태완 의원이 선출됨."
898,"의사일정 제3항, 밥쌀용 쌀 수입 반대 결의안을 상정합니다. 대표발의하신 이상정 의...",밥쌀용 쌀 수입 반대 결의안은 정부가 밥쌀용 쌀 3만 톤을 미국과 중국에서 수입하기...
1366,"의사일정 제4항, 음성군 지방공무원 정원조례 일부개정조례안, 의사일정 제5항, 음성...",음성군 지방공무원 정원조례 일부개정조례안은 전문위원의 직급과 정수 기준이 개정되어 ...
1927,"의사일정 제3항, 2006년도 제2회 세입·세출 추가경정예산안 및 2006년도 기금...","2006년도 제2회 세입, 세출 추가경정예산안 및 2006년도 기금운용계획변경계획안..."
1177,산림녹지과장입니다. 2018년도 행정사무감사 시 시정ㆍ건의사항에 대한 조치결과 및...,만세기념비 등이 있는 오류리 공원의 화장실을 관리하여 주민들이 실제로 이용할 수 있...


In [ ]:
df_val.head()

,text_concat,label
754,"의사일정 제2항, 2001년도행정사무감사결과보고서채택의건을 상정합니다. 행정사무감사...",2001년도 행정사무감사 결과보고서를 채택함.
1219,의사일정 제8항 1998공유재산관리계획변경승인안을 상정합니다. 산림과장께서는 본 승...,1998년도 공유재산 관리계획 변경승인안이 가결됨.
1980,의석을 정돈해 주시기 바랍니다. 성원이 되었으므로 제42회 청주시의회(임시회) 제...,제42회 청주시의회(임시회) 제1차 본회의 개의 선포.
591,의사일정 제3항 음성군세감면조례중개정조례안을 상정합니다. 재무과장께서는 나오셔서 본...,"음성군세감면조례중개정조례안은 농외 소득원 개발, 재래시장 재개발, 재건축 등 국가정..."
1900,다음은 의사일정 제7항 휴회의 건을 상정합니다. 본 건은 상임위원회 및 특별위원회...,11월 28일부터 12월 10일까지 13일간 휴회가 가결됨.


In [ ]:
df_test.head()

,text_concat,label
2420,의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시 ...,제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...
2866,"의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다. 행정과장님...",음성군 지역치안협의회 조례 일부개정조례안은 지역치안협의회에 보조금 지원이 용이하도록...
1724,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.
1124,다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다. ...,예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...
975,"다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3항 ...",완주군 지방공무원정원 조례 일부개정조례안은 부동산 실거래가 신고제 등 토지관련 업무...


In [ ]:
# 데이터 전처리 / tokenizer Issue 

# 테스트셋의 문단 길이 확인 
lst_up = []
for i in df_test.index :
  text = df_test['text_concat'][i]
  if len(text) >= 1024:
    lst_up.append(i)

print(len(lst_up))

# 테스트셋 중 글자수 1024 이상 데이터 제외 > 추후 데이터 전처리(1024 이하로 분할)
# df_test 2226 > IndexError: index out of range in self > embedding error 
# 넣은 문단이 너무 길어서 발생 / pretrained tokenizer 대부분이 1024 이하로 한정되어있기 때문에 그에 맞춰서 전처리 필요 

df_test_new = df_test[~df_test.index.isin(lst_up)]
df_test_new.shape

# 가공된 테스트셋의 문단 길이 이상치 확인 
lst_up = []
for i in df_test_new.index :
  text = df_test_new['text_concat'][i]
  if len(text) >= 1024:
    lst_up.append(i)

len(lst_up)

0

###### **DL Comparison**

In [ ]:
# model - 직접 학습 
# tokenizer - gogamza/kobart-summarization
# 소요시간 약 20분 (개당 3~5초)

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/project4/RawData/KoBART-summarization/kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')


lst_summary = []
for index in list(df_test_new.index):
  print(index) #현재 진행위치 기록용 
  text = df_test_new.loc[index, 'text_concat']
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
  summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  lst_summary.append((index, text, summary))

print(lst_summary[0])


Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

2420
1724
1124
2945
2560
891
1929
794
829
1272
1827
2513
2761
677
1356
193
441
63
2205
2471
2699
692
425
2373
2153
2040
1566
2355
1001
212
2485
768
1735
2421
1172
52
1794
1371
45
2843
1556
179
103
511
0
1920
1834
1271
889
357
335
324
560
2231
2044
1211
1638
415
1386
821
1737
2345
549
204
537
195
2828
1610
51
2671
2663
700
1393
2379
608
1084
1987
2202
532
1190
2200
1725
1604
609
2540
1064
2697
2372
67
2368
197
1901
745
702
2237
1346
1898
2807
1388
2595
1507
2409
2549
1009
1389
1993
2122
1105
1349
80
32
249
1255
1329
1418
88
1419
2307
2855
2840
1577
1252
2634
2902
790
1284
142
2700
1447
1674
1761
2717
1642
2923
2177
2466
2804
178
185
2755
259
44
855
2264
1068
2544
1409
712
2893
476
1145
698
490
254
1241
619
2625
1242
1247
30
2876
325
2972
1817
1134
1723
2436
299
2696
2303
73
1665
943
1350
1778
765
2441
2328
2183
1525
2432
1279
242
538
2545
1931
1040
1578
1203
773
2645
2809
1139
2141
976
570
2362
2916
2055
2191
1047
796
2732
2740
1854
1414
1474
1488
1876
70
2251
1299
1560
56
1579
1091
143

In [ ]:
df_test_mymodel_gogamza_summary = pd.DataFrame(lst_summary, columns=['index','text','summary'])
df_test_mymodel_gogamza_summary.to_csv('df_test_mymodel_gogamza_summary.csv', encoding='utf-8')

In [ ]:
# model - 직접 학습 
# tokenizer - gogamza/kobart-base-v2  == gogamza/kobart-summarization
## 동일 사람이 제작한 토크나이저에 따른 결과 차이 없었음 
# 소요시간 / 16분 

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/project4/RawData/KoBART-summarization/kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')


lst_summary_v2 = []
for index in list(df_test_new.index):
  print(index) #현재 진행위치 기록용 
  text = df_test_new.loc[index, 'text_concat']
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
  summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  lst_summary_v2.append((index, text, summary))

print(lst_summary_v2[0])

df_test_mymodel_gogamza_v2 = pd.DataFrame(lst_summary_v2, columns=['index','text','summary'])
df_test_mymodel_gogamza_v2.to_csv('df_test_mymodel_gogamza_v2.csv', encoding='utf-8')

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

2420
1724
1124
2945
2560
891
1929
794
829
1272
1827
2513
2761
677
1356
193
441
63
2205
2471
2699
692
425
2373
2153
2040
1566
2355
1001
212
2485
768
1735
2421
1172
52
1794
1371
45
2843
1556
179
103
511
0
1920
1834
1271
889
357
335
324
560
2231
2044
1211
1638
415
1386
821
1737
2345
549
204
537
195
2828
1610
51
2671
2663
700
1393
2379
608
1084
1987
2202
532
1190
2200
1725
1604
609
2540
1064
2697
2372
67
2368
197
1901
745
702
2237
1346
1898
2807
1388
2595
1507
2409
2549
1009
1389
1993
2122
1105
1349
80
32
249
1255
1329
1418
88
1419
2307
2855
2840
1577
1252
2634
2902
790
1284
142
2700
1447
1674
1761
2717
1642
2923
2177
2466
2804
178
185
2755
259
44
855
2264
1068
2544
1409
712
2893
476
1145
698
490
254
1241
619
2625
1242
1247
30
2876
325
2972
1817
1134
1723
2436
299
2696
2303
73
1665
943
1350
1778
765
2441
2328
2183
1525
2432
1279
242
538
2545
1931
1040
1578
1203
773
2645
2809
1139
2141
976
570
2362
2916
2055
2191
1047
796
2732
2740
1854
1414
1474
1488
1876
70
2251
1299
1560
56
1579
1091
143

In [ ]:
# model - gogamza/kobart-summarization
# tokenizer - gogamza/kobart-summarization
## premodel 사용 시 결과 비교 
# 소요시간 24분 

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')


lst_pre_summary = []
for index in list(df_test_new.index):
  print(index) #현재 진행위치 기록용 
  text = df_test_new.loc[index, 'text_concat']
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=128,  eos_token_id=1)
  summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  lst_pre_summary.append((index, text, summary))
  print(summary)

print(lst_pre_summary[0])

df_test_premodel_gogamza_summary = pd.DataFrame(lst_pre_summary, columns=['index','text','summary'])
df_test_premodel_gogamza_summary.to_csv('df_test_premodel_gogamza_summary.csv', encoding='utf-8')

2420
7월 3일 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5항으로 상정하여 처리하고자 하는 의사일정변경의 건이 가결되었음을 선포하며 잠시 정회를 선포한다.
1724
지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 따라 제102회 완주군의회 임시회 회의록에 서명하실 의원님은 박웅배 의원님과 김순길 의원님을 선임하고자 하는데 의원 여러분 이의 없으십니까?(『없습니다』하는 의원 있음)그러면 두 분 의원님께서는 본 회기동안 수고하여 주시기 바랍니다. 
1124
완주군의회 예산결산 특별위원회는 2011년도 완주군 세입ᆞ세출결산 및 예비비지출 승인안과 2012년도부터 2013년도 예산안 등 관련 안건을 종합적이고 객관적으로 심사하기 위해 ‘지방자치법」제56조와「완주군의회 위원회 조례」제7조 제2항에 의거 구성하려는 것으로 특별위원회 명칭은 ‘완주군의회 예산결산 특별위원회’로 하고, 구성인원수는 의장님을 제외한 9인으로 하며, 존속기간은 2012년 7월 10일부터 2013년 7월 9일까지로 하고자 한다.
2945
제208회 완주군의회 임시회 제1차 본회의에서 제208회 완주군의회 임시회 임시회 임시회 제1차 본회의를 가결하였습니다.
2560
성원이 되었으므로 지금부터 제75회 음성군의회 임시회 제2차 본회의를 개의하겠습니다. 
891
완주군의회 김재남 의원은 군정에 관한 질문과 답변을 통해 군민의 의사를 군정에 반영시키고자 지방자치법 제37조 제1항 및 완주군의회 회의규칙 제66조 제1항의 규정에 따라 '98년 4월 8일 군정에 관한 질문을 하고, '98년 4월 10일 군정질문에 관한 답변을 듣기 위하여 완주군수, 공보담당관, 내무과장, 사회복지과장, 환경보호과장, 지역경제과장, 도시과장, 도시과장, 농촌지도소장, 보건소장의 출석을 각각 요구하는' 관계공무원 출석요구의 건에 대하여 제안설명을 드리겠습니다.
1929
의사일정 제5항, 제172회 임시회 휴회의 건은 금번 회기 중 오늘부터 예산결산특별위원회 활동을 하고자 하는 것으로 휴회의 

In [ ]:
# 옵션 1
# model - 직접 학습 
# tokenizer - digit82/kobart-summarization
## 다른 사람에 제작한 tokenizer와의 비교 
# 소요시간 17분

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/project4/RawData/KoBART-summarization/kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')


lst_summary_digit = []
for index in list(df_test_new.index):
  print(index) #현재 진행위치 기록용 
  text = df_test_new.loc[index, 'text_concat']
  label = df_test_new.loc[index, 'label']
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
  summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  lst_summary_digit.append((index, text, summary, label))

print(lst_summary_digit[0])

df_test_mymodel_digit_summary= pd.DataFrame(lst_summary_digit, columns=['index','text','summary','label'])
df_test_mymodel_digit_summary.to_csv('df_test_mymodel_digit_summary.csv', encoding='utf-8')

Downloading:   0%|          | 0.00/109 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/295 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

2420
1724
1124
2945
2560
891
1929
794
829
1272
1827
2513
2761
677
1356
193
441
63
2205
2471
2699
692
425
2373
2153
2040
1566
2355
1001
212
2485
768
1735
2421
1172
52
1794
1371
45
2843
1556
179
103
511
0
1920
1834
1271
889
357
335
324
560
2231
2044
1211
1638
415
1386
821
1737
2345
549
204
537
195
2828
1610
51
2671
2663
700
1393
2379
608
1084
1987
2202
532
1190
2200
1725
1604
609
2540
1064
2697
2372
67
2368
197
1901
745
702
2237
1346
1898
2807
1388
2595
1507
2409
2549
1009
1389
1993
2122
1105
1349
80
32
249
1255
1329
1418
88
1419
2307
2855
2840
1577
1252
2634
2902
790
1284
142
2700
1447
1674
1761
2717
1642
2923
2177
2466
2804
178
185
2755
259
44
855
2264
1068
2544
1409
712
2893
476
1145
698
490
254
1241
619
2625
1242
1247
30
2876
325
2972
1817
1134
1723
2436
299
2696
2303
73
1665
943
1350
1778
765
2441
2328
2183
1525
2432
1279
242
538
2545
1931
1040
1578
1203
773
2645
2809
1139
2141
976
570
2362
2916
2055
2191
1047
796
2732
2740
1854
1414
1474
1488
1876
70
2251
1299
1560
56
1579
1091
143

In [ ]:
# 옵션 2 > 진행X 
# model - digit82/kobart-summarization 
# tokenizer - digit82/kobart-summarization 
## 다른 premodel 사용결과 비교 / 다른 사람이 제작 
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

text = df_test['text_concat'][2420]

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=128,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

'당초 7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5항으로 상정하여 처리하고자 하는데 의원여러분 이의 없으십니까?(『없습니다』하는 의원 있음. )이의가 없으므로 가결되었음을 선포한다.'

In [ ]:
# 옵션 3 > 진행 X 
# model - gogamza/kobart-base-v2
# tokenizer - gogamza/kobart-base-v2
## 다른 premodel 사용결과 비교 / summary 전용이 아닌 kobart
# 소요시간 / 1분당 3건 > 4시간

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')


lst_summary = []
for index in list(df_test['text_concat'].index):
  print(index)
  text = df_test.loc[index, 'text_concat']
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

  summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=128,  eos_token_id=1)
  summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
  lst_summary.append((text, summary))

print(lst_summary[0])

Downloading:   0%|          | 0.00/473M [00:00<?, ?B/s]

2420
2866
1724
1124
975
2945
468
2560
891
2226


IndexError: ignored

###### **KoBART Error Solution**

In [ ]:
# model input 형태 확인 
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

from transformers import PreTrainedTokenizerFast, BartModel


tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

text = str(df['text'][0][0] + df['text'][0][1])
# 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제1차 본회의 개의를 선포합니다. 
# 먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 주시기 바랍니다.
# 의사팀장 이은미 입니다. 의회관련 사항을 보고 드리겠습니다. 먼저 제207회 완주군의회 임시회 소집경위를 보고 드리겠습니다. 
# 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 
# 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 의안접수 및 회부사항입니다. 
# 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 보호 
# 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 
# 다음은 완주군수로부터 제출된 안건입니다. 
# 삼례문화예술촌 설치 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 
# 자세한 내용은 배부해 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다.


raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=18,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)


# 512
# 제207회 완주군의회 임시회 임시회 제1차 본회의 개의를 선포하며 의안접수 및 회부사항, 삼례문화예술촌 설치 및 운영조례 일부개정 조례안, 
# 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로 해당 상임의원회로 회부하였다.

# 126 
# 제207회 완주군의회 임시회 임시회 제1차 본회의 개의를 선포하며 의안접수 및 회부사항, 삼례문화예술촌 설치 및 운영조례 일부개정 조례안, 
# 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로 해당 상임의원회로 회부하였다.

# max_length = 18
# 제207회 완주군의회 임시회 제1차 본회의 개의를 선포

'제207회 완주군의회 임시회 제1차 본회의 개의를 선포'

In [ ]:
# model - 직접 학습 , tokenizer - digit82/kobart-summarization 사용 

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/project4/RawData/KoBART-summarization/kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')

text = df_test['text_concat'][2420]

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)


'7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5항으로 가결됨.'

In [ ]:
# 에러 실험 
# digit82/kobart-summarization
# gogamza/kobart-base-v2
# gogamza/kobart-summarization
# tokenizer 1024 이하인 문단에 대해서만 요약 가능 > 논문용은 longformer 사용

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/project4/RawData/KoBART-summarization/kobart_summary')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')

text = df_test['text_concat'][2226] #len = 1294 

# 문단 길이가 긴 문장에 대해 자름 
raw_input_ids = tokenizer.encode(text)
if len(raw_input_ids) >= 512:
  raw_input_ids1 = raw_input_ids[:1024]

input_ids = [tokenizer.bos_token_id] + raw_input_ids1 + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


'예산결산특별위원회에서 지난 3월 13일부터 3월 18일까지 심사한 2019년도 제1회 세입 세출 추가경정예산안 및 기금운용계획 변경안 승인의 건을 일괄 상정함.'

In [ ]:
# gogamza/kobart-base-v1 사용
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

from transformers import PreTrainedTokenizerFast, BartModel


tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')

text = df_test['text_concat'][2420]

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

''

In [ ]:
#digit82/kobart-summarization 사용 
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

text = str(df['text'][0][0] + df['text'][0][1])
# 의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 제207회 완주군의회 임시회 제1차 본회의 개의를 선포합니다. 
# 먼저 의사팀장으로부터 의회 관련 사항에 대한 보고가 있겠습니다. 의사팀장은 보고하여 주시기 바랍니다.
# 의사팀장 이은미 입니다. 의회관련 사항을 보고 드리겠습니다. 먼저 제207회 완주군의회 임시회 소집경위를 보고 드리겠습니다. 
# 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 
# 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 되었습니다. 다음은 의안접수 및 회부사항입니다. 
# 먼저 의원발의 안건입니다. 완주군의회 의원 의정활동비등 지급에 관한 조례 일부개정 조례안, 완주군 향토문화유산 보호 
# 및 관리 조례안, 완주군 싸움소 육성 지원에 관한 조례안 등 총 3건으로, 해당 상임의원회로 회부 하였습니다. 
# 다음은 완주군수로부터 제출된 안건입니다. 
# 삼례문화예술촌 설치 및 운영조례 일부개정 조례안 등 총 40건으로 해당 상임의원회로 회부하였습니다. 
# 자세한 내용은 배부해 드린 유인물을 참조하여 주시기 바랍니다. 이상 보고를 마치겠습니다.


raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

# max_length 
# 512
# 지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 
# 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 된 제207회 완주군의회 임시회 제1차 본회의 개의를 선포했다.


'지방자치법 제45조 제2항의 규정에 의거 서남용 의원님 외 4인의 의원님으로부터 임시회 집회 요구가 있어 지난 8월6일 집회 공고를 하고 오늘 집회를 갖게 된 제207회 완주군의회 임시회 제1차 본회의 개의를 선포했다.'

- https://github.com/SKT-AI/KoBART KoBART 기본
- https://huggingface.co/spaces/gogamza/kobart-summarization/blob/main/app.py KoBART 공식Demo
- https://huggingface.co/gogamza/kobart-base-v1/blob/main/README.md pretrained call
- https://younghwani.github.io/posts/kobart-summary-3/ KoBART 데모 1
- http://blog.ju-ing.co.kr/posts/KoBART-summarization/ KoBART 데모 2
- https://github.com/seujung/KoBART-summarization KoBART 데모 3 seujung

In [ ]:
# ToyText 

# text = """
# 1일 오후 9시까지 최소 20만3220명이 코로나19에 신규 확진됐다. 또다시 동시간대 최다 기록으로, 사상 처음 20만명대에 진입했다.
# 방역 당국과 서울시 등 각 지방자치단체에 따르면 이날 0시부터 오후 9시까지 전국 신규 확진자는 총 20만3220명으로 집계됐다.
# 국내 신규 확진자 수가 20만명대를 넘어선 것은 이번이 처음이다.
# 동시간대 최다 기록은 지난 23일 오후 9시 기준 16만1389명이었는데, 이를 무려 4만1831명이나 웃돌았다. 전날 같은 시간 기록한 13만3481명보다도 6만9739명 많다.
# 확진자 폭증은 3시간 전인 오후 6시 집계에서도 예견됐다.
# 오후 6시까지 최소 17만8603명이 신규 확진돼 동시간대 최다 기록(24일 13만8419명)을 갈아치운 데 이어 이미 직전 0시 기준 역대 최다 기록도 넘어섰다. 역대 최다 기록은 지난 23일 0시 기준 17만1451명이었다.
# 17개 지자체별로 보면 서울 4만6938명, 경기 6만7322명, 인천 1만985명 등 수도권이 12만5245명으로 전체의 61.6%를 차지했다. 서울과 경기는 모두 동시간대 기준 최다로, 처음으로 각각 4만명과 6만명을 넘어섰다.
# 비수도권에서는 7만7975명(38.3%)이 발생했다. 제주를 제외한 나머지 지역에서 모두 동시간대 최다를 새로 썼다.
# 부산 1만890명, 경남 9909명, 대구 6900명, 경북 6977명, 충남 5900명, 대전 5292명, 전북 5150명, 울산 5141명, 광주 5130명, 전남 4996명, 강원 4932명, 충북 3845명, 제주 1513명, 세종 1400명이다.
# 집계를 마감하는 자정까지 시간이 남아있는 만큼 2일 0시 기준으로 발표될 신규 확진자 수는 이보다 더 늘어날 수 있다. 이에 따라 최종 집계되는 확진자 수는 21만명 안팎을 기록할 수 있을 전망이다.
# 한편 전날 하루 선별진료소에서 이뤄진 검사는 70만8763건으로 검사 양성률은 40.5%다. 양성률이 40%를 넘은 것은 이번이 처음이다. 확산세가 계속 거세질 수 있다는 얘기다.
# 이날 0시 기준 신규 확진자는 13만8993명이었다. 이틀 연속 13만명대를 이어갔다.
# """

# text = text.replace('\n', ' ')

#### **Evaluation**

In [ ]:
# 테스트셋의 문단 길이 확인 
lst_up = []
for i in df_test.index :
  text = df_test['text_concat'][i]
  if len(text) >= 1024:
    lst_up.append(i)

len(lst_up)

# 테스트셋 중 글자수 1024 이상 데이터 제외 > 추후 데이터 전처리(1024 이하로 분할)
# df_test 2226 > IndexError: index out of range in self > embedding error 
# 넣은 문단이 너무 길어서 발생 / pretrained tokenizer 대부분이 1024 이하로 한정되어있기 때문에 그에 맞춰서 전처리 필요 

df_test_new = df_test[~df_test.index.isin(lst_up)]
df_test_new.shape

# 가공된 테스트셋의 문단 길이 이상치 확인 
lst_up = []
for i in df_test_new.index :
  text = df_test_new['text_concat'][i]
  if len(text) >= 1024:
    lst_up.append(i)

len(lst_up)

0

In [ ]:
df_test_new.shape

(458, 2)

In [ ]:
df_test_new.to_csv('df_test_label.csv', encoding='utf-8')

In [ ]:
# rouge-2 score 
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


###### **Rouge Score**
- 텍스트 자동 요약, 기계 번역 등을 평가하는 지표
- 모델이 생성한 요약본 혹은 번역본을 사람이 미리 만들어 놓은 참조본과 대조해 성능 점수(precision과 recall, f1-score)를 계산
- ROUGE-N: unigram, bigram, trigram 등 문장 간 중복되는 n-gram을 비교하는 지표
- ROUGE-L: LCS 기법을 이용해 최장 길이로 매칭되는 문자열을 측정. ROUGE-2와 같이 단어들의 연속적 매칭을 요구하지 않고, 어떻게든 문자열 내에서 발생하는 매칭을 측정하기 때문에 보다 유연한 성능 비교가 가능

In [ ]:
%pwd

'/content/drive/MyDrive/project4/RawData'

In [ ]:
%cd KoBART-summarization

/content/drive/MyDrive/project4/RawData/KoBART-summarization


In [ ]:
# 데이터 불러오기

df_test_mymodel_gogamza_summary = pd.read_csv('df_test_mymodel_gogamza_summary.csv', index_col=0)
df_test_mymodel_digit_summary = pd.read_csv('df_test_mymodel_digit_summary.csv', index_col=0)
df_test_premodel_gogamza_summary = pd.read_csv('df_test_premodel_gogamza_summary.csv', index_col=0)
df_test_textrank = pd.read_csv('textrank.csv', index_col=0)


# 토크나이저 비교용 
# df_test_mymodel_gogamza_summary
# df_test_mymodel_digit_summary

# # 모델학습 비교용 
# df_test_premodel_gogamza_summary

# c.f. df_test_mymodel_gogamza_v2의 결과는 df_test_mymodel_gogamza_summary와 동일함

In [ ]:
df_test_mymodel_gogamza_summary.shape, df_test_new.shape

((458, 3), (458, 2))

In [ ]:
df_test_new.head()

,text_concat,label
2420,의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시 ...,제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...
1724,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.
1124,다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다. ...,예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...
2945,"다음은 의사일정 제5항, 본회의 휴회의 건을 상정합니다. 상임위원회 등 의정활동을...",10월 14일부터 10월 25일까지 12일간 휴회가 가결됨. 제2차 본회의는 10월...
2560,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 제75회 음성군의회 임...,제75회 음성군의회 임시회 제2차 본회의 개의 선포.


In [ ]:
df_test_mymodel_digit_summary.head()

,index,text,summary,label
0,2420,의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시 ...,7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5...,제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...
1,1724,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원이 선임됨.,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.
2,1124,다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다. ...,"완주군의회 예산결산 특별위원회 구성 결의안은 2011년도 완주군 세입, 세출결산 및...",예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...
3,2945,"다음은 의사일정 제5항, 본회의 휴회의 건을 상정합니다. 상임위원회 등 의정활동을...",10월 14일부터 10월 25일까지 12일간 휴회가 가결됨. 제2차 본회의는 10월...,10월 14일부터 10월 25일까지 12일간 휴회가 가결됨. 제2차 본회의는 10월...
4,2560,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 제75회 음성군의회 임...,제75회 음성군의회 임시회 제2차 본회의 개의 선포.,제75회 음성군의회 임시회 제2차 본회의 개의 선포.


In [ ]:
df_test_mymodel_gogamza_summary.head()

,index,text,summary
0,2420,의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시 ...,7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5...
1,1724,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원이 선임됨.
2,1124,다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다. ...,"완주군의회 예산결산 특별위원회 구성 결의안은 2011년도 완주군 세입, 세출결산 및..."
3,2945,"다음은 의사일정 제5항, 본회의 휴회의 건을 상정합니다. 상임위원회 등 의정활동을...",10월 14일부터 10월 25일까지 12일간 휴회가 가결됨. 제2차 본회의는 10월...
4,2560,의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 제75회 음성군의회 임...,제75회 음성군의회 임시회 제2차 본회의 개의 선포.


In [ ]:
df_test_textrank.head()

,id,agenda,text,label,text_concat,text_split,len_label,textrank,summary
2420,1377,AGENDA_4,['의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시...,제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...,의사일정 제3항 의사일정변경의 건을 상정합니다. 당초 7월 3일 제2차 본회의시 ...,"['의사일정 제3항 의사일정변경의 건을 상정합니다.', '당초 7월 3일 제2차 본...",55,"[(1, 1.6682934603175785, '당초 7월 3일 제2차 본회의시 상임...",당초 7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항...
2866,1444,AGENDA_4,"['의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다.', '...",음성군 지역치안협의회 조례 일부개정조례안은 지역치안협의회에 보조금 지원이 용이하도록...,"의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다. 행정과장님...","['의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 상정합니다.', '...",78,"[(29, 1.4717813659629004, '의사일정 제3항, 음성군 지역치안협...","의사일정 제3항, 음성군 지역치안협의회 조례 일부개정조례안을 원안대로 의결하고자 하..."
1724,1272,AGENDA_5,['다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제10...,제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.,다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...,['다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제10...,47,"[(0, 1.2249777916398097, '다음은 지방자치법 제64조제2항과 회...",다음은 지방자치법 제64조제2항과 회의규칙 제46조제1항의 규정에 의하여 제102회...
1124,1183,AGENDA_4,['다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다....,예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...,다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다. ...,['다음은 의사일정 제3항 완주군의회 예산결산 특별위원회 구성 결의안을 상정합니다....,164,"[(4, 1.4170457588599894, '존경하는 의장님, 그리고 동료 의원 ...","존경하는 의장님, 그리고 동료 의원 여러분!완주군의회 예산결산 특별위원회 구성 결의..."
975,1157,AGENDA_3,"['다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3...",완주군 지방공무원정원 조례 일부개정조례안은 부동산 실거래가 신고제 등 토지관련 업무...,"다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3항 ...","['다음은 의사일정 제2항 완주군 지방공무원정원 조례 일부개정조례안, 의사일정 제3...",368,"[(18, 1.4308450362205591, '다음은 의사일정 제3항 완주군별정직...",다음은 의사일정 제3항 완주군별정직지방공무원의범위에관한조례 일부개정조례안에 대하여 ...


In [ ]:
df_test_mymodel_gogamza_summary['summary']

0      7월 3일 제2차 본회의시 상임위원회 위원 선임의 건과 상임위원장 선거를 4항과 5...
1        제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원이 선임됨.
2      완주군의회 예산결산 특별위원회 구성 결의안은 2011년도 완주군 세입, 세출결산 및...
3      10월 14일부터 10월 25일까지 12일간 휴회가 가결됨. 제2차 본회의는 10월...
4                          제75회 음성군의회 임시회 제2차 본회의 개의 선포.
                             ...                        
453            제154회 임시회 회의록 서명의원으로 이한철 의원과 윤병승 의원이 선출됨.
454                       4월 1일 인사이동에 따른 의회사무국 간부공무원 소개.
455    2014회계연도 결산검사위원으로 이인숙 의원, 김상철 의원, 임재평 의원, 국용호 ...
456    제253회 완주군의회 임시회 회기는 7월 14일부터 7월 23일까지 10일간으로 가결됨.
457      제252회 완주군의회 임시회 회의록 서명의원으로 김재천 의원과 이인숙 의원이 선출됨.
Name: summary, Length: 458, dtype: object

In [ ]:
df_test_new['label']

2420    제2차 본회의에 상임위원회 위원 선임의 건과 상임위원장 선거를 상정하기로 함. 해당...
1724      제102회 완주군의회 임시회 회의록 서명의원으로 박웅배 의원과 김순길 의원을 선임함.
1124    예산결산특별위원회위원은 의장을 제외한 9인으로 구성함. 특별위원회는 2012년 7월...
2945    10월 14일부터 10월 25일까지 12일간 휴회가 가결됨. 제2차 본회의는 10월...
2560                        제75회 음성군의회 임시회 제2차 본회의 개의 선포.
                              ...                        
2653            제154회 임시회 회의록 서명의원으로 이한철 의원과 윤병승 의원이 선출됨.
250                        4월 1일 인사이동에 따른 의회사무국 간부공무원 소개.
2976    2014회계연도 결산검사위원으로 대표의원 이인숙 의원, 전직 공무원 김상철, 임재평...
1757    제253회 완주군의회 임시회 회기는 7월 14일부터 7월 23일까지 10일간으로 가결됨.
2276      제252회 완주군의회 임시회 회의록 서명의원으로 김재천 의원과 이인숙 의원이 선출됨.
Name: label, Length: 458, dtype: object

In [ ]:
# rouge score 
# For multiple sentences 

from rouge import Rouge 

rouge = Rouge()

hyps = df_test_textrank['summary'].tolist()
refs = df_test_textrank['label'].tolist()

scores = rouge.get_scores(hyps, refs, avg=True)
print(scores)

{'rouge-1': {'r': 0.14628224946337942, 'p': 0.19493960712211672, 'f': 0.1503769391767267}, 'rouge-2': {'r': 0.09939165393619602, 'p': 0.13500117101072265, 'f': 0.10117654696255328}, 'rouge-l': {'r': 0.1449247895990249, 'p': 0.1932662923081871, 'f': 0.1490531318479023}}


In [ ]:
# rouge score / MyDL + gogamza/tokenizer 
# For multiple sentences 

from rouge import Rouge 

rouge = Rouge()

hyps = df_test_mymodel_gogamza_summary['summary'].tolist()
refs = df_test_new['label'].tolist()

scores = rouge.get_scores(hyps, refs, avg=True)
print(scores)

{'rouge-1': {'r': 0.8193468150794421, 'p': 0.8189892139559481, 'f': 0.8063565130948369}, 'rouge-2': {'r': 0.7365622623326448, 'p': 0.7404020430258149, 'f': 0.7279622576557658}, 'rouge-l': {'r': 0.8132259147248465, 'p': 0.8137673606740519, 'f': 0.8008201257564441}}


In [ ]:
# rouge score  / MyDL + digit/tokenizer
# For multiple sentences 

from rouge import Rouge 

rouge = Rouge()

hyps = df_test_mymodel_digit_summary['summary'].tolist()
refs = df_test_new['label'].tolist()

scores = rouge.get_scores(hyps, refs, avg=True)
print(scores)

{'rouge-1': {'r': 0.8193468150794421, 'p': 0.8189892139559481, 'f': 0.8063565130948369}, 'rouge-2': {'r': 0.7365622623326448, 'p': 0.7404020430258149, 'f': 0.7279622576557658}, 'rouge-l': {'r': 0.8132259147248465, 'p': 0.8137673606740519, 'f': 0.8008201257564441}}


In [ ]:
# rouge score  / preDL 
# For multiple sentences 

from rouge import Rouge 

rouge = Rouge()

hyps = df_test_premodel_gogamza_summary['summary'].tolist()
refs = df_test_new['label'].tolist()

scores = rouge.get_scores(hyps, refs, avg=True)
print(scores)

{'rouge-1': {'r': 0.5013521555892089, 'p': 0.2439318385731003, 'f': 0.31059669039267396}, 'rouge-2': {'r': 0.3638765317068805, 'p': 0.16386818124273733, 'f': 0.21357541903104657}, 'rouge-l': {'r': 0.48788962821374265, 'p': 0.23727582347553922, 'f': 0.302183379085435}}


- 성능지표 Rouge_score, bleu score 
  - https://wikidocs.net/31695 bleu score에 대하여 
  - https://huffon.github.io/2019/12/07/rouge/ rouge score에 대하여 
  - RDASS 논문(+계산과정)  https://kakaoenterprise.github.io/deepdive/210729 
  - RDASS에 대한 간단정리 https://velog.io/@idj7183/%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8-2%EC%9D%BC%EC%B0%A8 
  -  각종 지표에 대한 고찰 https://velog.io/@crosstar1228/NLPRouge-score-Summarization%EC%9D%98-%ED%8F%89%EA%B0%80-Metric
  - 한국어 텍스트 생성에 대해선, '순서'가 아닌 '조합'으로 성능을 판단할 수 있는 ROUGE-U, ROUGE-SU가 적절해보임

In [ ]:
# rouge score 
# For 1 sentence / ToyData

from rouge import Rouge 

rouge = Rouge()

# Define the generated and reference summaries
generated_summary = "A cat and a dog were playing in the park."
reference_summary = "A dog and a cat were playing in the park."

# Calculate the ROUGE-2 score
scores = rouge.get_scores(generated_summary, reference_summary, avg=True)
rouge_2_score = scores['rouge-2']

# Print the ROUGE-2 score
print(rouge_2_score)

{'r': 0.5555555555555556, 'p': 0.5555555555555556, 'f': 0.5555555505555557}


###### **bleu score**
- 번역에 알맞은 지표 

In [ ]:
# bleu score 측정 / 여러문장일때 
import nltk.translate.bleu_score as bleu

candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print('패키지 NLTK의 BLEU :',bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))

패키지 NLTK의 BLEU : 0.5045666840058485


In [ ]:
reference_sentence.split()

['A', 'dog', 'and', 'a', 'cat', 'were', 'playing', 'in', 'the', 'park.']

In [ ]:
generated_sentence.split()

['A', 'cat', 'and', 'a', 'dog', 'were', 'playing', 'in', 'the', 'park.']

In [ ]:
# 한문장일때 
# import nltk
from nltk.translate.bleu_score import sentence_bleu

# Define the generated sentence and reference sentence
generated_sentence = "A cat and a dog were playing in the park."
reference_sentence = "A dog and a cat were playing in the park."

# Calculate the BLEU score using the sentence_bleu function
bleu_score = sentence_bleu([reference_sentence.split()], generated_sentence.split())

# Print the BLEU score
print(bleu_score)


# 추후 이론 공부 필요한 지점
# Define the generated sentence and reference sentence
generated_sentence = "A cat and a dog were playing in the park."
reference_sentence = "A dog and a cat were playing in the park."

# Calculate the BLEU score using the sentence_bleu function
bleu_score = sentence_bleu([reference_sentence], generated_sentence)

# Print the BLEU score
print(bleu_score)

0.49393827371153703
0.9250077760445776


## **3. QA**
- 추후 제작 예정

#### **Installation**

- https://github.com/deepset-ai/haystack

In [ ]:
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-5z071et5
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-5z071et5
  Resolved https://github.com/deepset-ai/haystack.git to commit b9ab7b3ca2d548b3669038d837b83e6558fa8c6d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981

In [ ]:
!pip install farm-haystack

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/deepset-ai/haystack.git
%cd haystack
# !pip install --upgrade pip
# !pip install -e '.[all]'

Cloning into 'haystack'...
remote: Enumerating objects: 28864, done.
remote: Counting objects: 100% (668/668), done.
remote: Compressing objects: 100% (358/358), done.
remote: Total 28864 (delta 409), reused 503 (delta 301), pack-reused 28196
Receiving objects: 100% (28864/28864), 33.95 MiB | 13.31 MiB/s, done.
Resolving deltas: 100% (22071/22071), done.
Updating files: 100% (475/475), done.
/content/drive/MyDrive/project4/RawData/haystack


In [ ]:
%%bash
pip install farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 59.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.1
    Uninstalling numpy-1.24.1:
      Successfully uninstalled numpy-1.24.1


In [ ]:
!pip install --upgrade "urllib3==1.25.4" awscli

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.5/125.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 47.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.14
    Uninstalling urllib3-1.26.14:
      Successfully uninstalled urllib3-1.26.14
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.8.0 requires urllib3[socks]~=1.26, but you have urllib3 1.25.4 which is incompatible.
responses 0.

In [ ]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text #haystack.indexing -> haystack.preprocessor
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

ImportError: ignored

#### **Haystack**

##### **QA transformer**

- https://github.com/jonas-jun/haystack_search_engine/blob/main/Search_haystack.ipynb
- text to text transformer https://github.com/yeontaek/Text-to-Text-Transformer

- KoBERT를 이용한 기계 독해